# Aprendizado Profundo: CNNs

**Dataset:** On Vehicle Helmet Detection Dataset

**Problema:** Classificação de Segurança no Trânsito (Uso de Capacete)

O objetivo deste projeto é desenvolver um modelo de Redes Neurais Convolucionais (CNN) capaz de identificar automaticamente se motociclistas estão utilizando capacete de segurança.

Este problema é de alta relevância para sistemas de cidades inteligentes e segurança do trabalho, permitindo a automação da fiscalização e prevenção de acidentes fatais. O dataset utilizado contém imagens reais capturadas em ambiente de trânsito, divididas em classes de "Com Capacete" e "Sem Capacete".

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import kagglehub
import pathlib
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow import keras
from tensorflow.keras import layers, models, Input
from tensorflow.keras.models import Sequential

try:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    pass

print("Bibliotecas importadas !!!")

## 1. Processamento dos Dados
Nesta etapa, realizamos o carregamento e pré-processamento das imagens:
1. **Redimensionamento:** Todas as imagens foram padronizadas para o tamanho 128x128 pixels.
2. **Divisão:** O conjunto foi dividido em 80% para treino e 20% para validação.
3. **Batching:** As imagens são processadas em lotes (batches) de 32.

In [ ]:
print("Baixando/Verificando o dataset do Kaggle...")
path_root = kagglehub.dataset_download("rajeevsekar21/on-vehicle-helmet-detection-dataset")

data_dir = pathlib.Path(path_root) / 'Helmet_Dataset'

print(f"Diretório: {data_dir}")

try:
    print(f"Classes (pastas) encontradas: {os.listdir(data_dir)}")
except Exception as e:
    print(f"Erro ao ler diretório: {e}")

batch_size = 32
img_height = 128
img_width = 128

print("\n--- Carregando treino...")
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

print("--- Carregando validação...")
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(f"\n --- CLASSES FINAIS: {class_names} ---")

Visualizando 9 imagens do dataset:

In [ ]:
plt.figure(figsize=(12, 6))
for images, labels in train_ds.take(1):
  for i in range(8):
    ax = plt.subplot(2, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
plt.show()

## 3. Treinamento e Avaliação

### Arquitetura da Rede
Utilizamos uma arquitetura CNN Sequencial com as seguintes características:
* **Rescaling:** Normalização dos pixels (0 a 1).
* **Data Augmentation:** Rotação e espelhamento aleatórios para evitar overfitting.
* **Camadas Convolucionais (Conv2D):** 3 blocos para extração de características visuais.
* **MaxPooling:** Para redução de dimensionalidade.
* **Dense/Dropout:** Camadas finais para classificação e regularização.

In [ ]:
num_classes = len(class_names)

model = models.Sequential([
  # --- A CORREÇÃO ESTÁ AQUI EMBAIXO ---
  # Em vez de colocar input_shape no Rescaling, criamos uma camada Input explícita
  Input(shape=(img_height, img_width, 3)),
  
  layers.Rescaling(1./255), # Normalização (sem o input_shape aqui)

  # O resto continua igual
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),

  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
  
  layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.summary()

In [ ]:
epochs = 15 

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

print("Treinamento concluído !!")
acc_final = history.history['val_accuracy'][-1]
print(f"Acuracia: {acc_final*100:.0f}% ({acc_final:.6f})")

## 4. Resultados Obtidos
Abaixo apresentamos os gráficos de desempenho do modelo.
* **Acurácia:** Indica a porcentagem de acertos.
* **Loss (Perda):** Indica o erro do modelo (quanto menor, melhor).

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(12, 5))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Treino: Acurácia')
plt.plot(epochs_range, val_acc, label='Validação: Acurácia')
plt.legend(loc='lower right')
plt.title('Evolução da Acurácia')
plt.xlabel('Épocas')
plt.ylabel('Acurácia (0 a 1)')
plt.grid(True)

# Erro (Loss)
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Treino: Erro')
plt.plot(epochs_range, val_loss, label='Validação: Erro')
plt.legend(loc='upper right')
plt.title('Redução do Erro (Loss)')
plt.xlabel('Épocas')
plt.ylabel('Erro')
plt.grid(True)

plt.show()

In [ ]:
print("Gerando previsões para calcular métricas...")
y_true = []
y_pred = []

for images, labels in val_ds:
    predictions = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Matriz de Confusão
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Previsão do Modelo')
plt.ylabel('Real (Verdadeiro)')
plt.title('Matriz de Confusão')
plt.show()

# Precisão, Recall, F1-Score
print("\n--- Relatório de Classificação ---")
print(classification_report(y_true, y_pred, target_names=class_names))

## 5. Conclusão

Neste projeto, desenvolvemos uma Rede Neural Convolucional (CNN) para auxiliar na segurança do trânsito através da detecção automática do uso de capacetes.

**Resultados Obtidos:**
O modelo demonstrou um desempenho satisfatório, atingindo uma acurácia de validação próxima a **87%**. Os gráficos de treinamento indicam que houve convergência adequada, com as curvas de treino e validação seguindo tendências similares, o que sugere que as técnicas de regularização (Dropout e Data Augmentation) foram eficazes em evitar o overfitting, mesmo com um dataset limitado.

**Análise das Métricas:**
A matriz de confusão nos permitiu identificar que o modelo possui alta taxa de acerto na distinção entre motociclistas com e sem capacete. O F1-Score equilibrado reforça a robustez do classificador.

**Próximos Passos:**
Para trabalhos futuros, o desempenho poderia ser aprimorado com:
1.  Aumento do dataset (coleta de mais imagens).
2.  Uso de *Transfer Learning* com redes pré-treinadas (como VGG16 ou ResNet).
3.  Implementação de detecção de objetos (YOLO) para localizar múltiplos motociclistas na mesma cena.